<a href="https://colab.research.google.com/github/J3rmed/ai4eng_2025_2/blob/main/03-modelo%20con%20preprocesado%20y%20LightGBM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 03 - modelo con LightGBM

Un modelo de clasificación utilizando LightGBM. Seguiendo el siguiente flujo de trabajo: cargar los datos, aplicar el preprocesado, evaluar y entrenar el modelo, finalmente, procesar los datos de prueba y generar un archivo de submission para Kaggle.

In [1]:
!pip install lightgbm

import numpy as np
import pandas as pd
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
import warnings

# Importante: Cargar el kaggle token
import os
os.environ['KAGGLE_CONFIG_DIR'] = '.'
!chmod 600 ./kaggle.json
!kaggle competitions download -c udea-ai-4-eng-20252-pruebas-saber-pro-colombia

  0% 0.00/29.9M [00:00<?, ?B/s]
100% 29.9M/29.9M [00:00<00:00, 1.05GB/s]


In [2]:
!unzip udea*.zip > /dev/null

In [3]:
!wc *.csv

   296787    296787   4716673 submission_example.csv
   296787   4565553  59185238 test.csv
   692501  10666231 143732437 train.csv
  1286075  15528571 207634348 total


In [14]:
z_train = pd.read_csv("train.csv")
z_test = pd.read_csv("test.csv")

print("Datos de entrenamiento cargados:", z_train.shape)
print("Datos de prueba cargados:", z_test.shape)

Datos de entrenamiento cargados: (692500, 21)
Datos de prueba cargados: (296786, 20)


In [15]:
# Definición de Columnas
# Variable objetivo que queremos predecir
target_col = 'RENDIMIENTO_GLOBAL'

columnas_a_eliminar = ['F_TIENEINTERNET.1', 'E_PRIVADO_LIBERTAD','F_TIENELAVADORA','E_PRGM_DEPARTAMENTO']

X = z_train.drop(columns=[target_col] + columnas_a_eliminar)
y = z_train[target_col]

# Identificar las columnas numéricas y categóricas que vamos a procesar
numeric_features = X.select_dtypes(include=['int64', 'float64']).columns.drop('ID').tolist()
categorical_features = X.select_dtypes(include=['object']).columns.tolist()

print(f"Features numéricas: {numeric_features}")
print(f"Features categóricas: {categorical_features}")

# Pipeline para variables numéricas: imputa con la mediana y luego escala

numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])

# Pipeline para variables categóricas: imputa con la moda y luego aplica One-Hot Encoding

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore', sparse_output=False))
])

# Creación del Preprocesador con ColumnTransformer

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ],
    remainder='passthrough'
)

# Codificación de la Variable Objetivo
# Mapeamos las etiquetas de texto a números
mapa_rendimiento = {'bajo': 0, 'medio-bajo': 1, 'medio-alto': 2, 'alto': 3}
y_encoded = y.map(mapa_rendimiento)

print(" Preprocesamiento definido.")

Features numéricas: ['PERIODO_ACADEMICO', 'INDICADOR_1', 'INDICADOR_2', 'INDICADOR_3', 'INDICADOR_4']
Features categóricas: ['E_PRGM_ACADEMICO', 'E_VALORMATRICULAUNIVERSIDAD', 'E_HORASSEMANATRABAJA', 'F_ESTRATOVIVIENDA', 'F_TIENEINTERNET', 'F_EDUCACIONPADRE', 'F_TIENEAUTOMOVIL', 'E_PAGOMATRICULAPROPIO', 'F_TIENECOMPUTADOR', 'F_EDUCACIONMADRE']
 Preprocesamiento definido.


In [16]:
# (Preprocesador + Modelo)
# Unimos el preprocesador con el clasificador Lightgbmm
full_pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', lgb.LGBMClassifier(random_state=42))
])


# Entrenamiento

full_pipeline.fit(X, y_encoded)
print("Entrenamiento finalizado.")

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.068394 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 2774
[LightGBM] [Info] Number of data points in the train set: 692500, number of used features: 867
[LightGBM] [Info] Start training from score -1.387092
[LightGBM] [Info] Start training from score -1.391216
[LightGBM] [Info] Start training from score -1.395031
[LightGBM] [Info] Start training from score -1.371991
Entrenamiento finalizado.


In [17]:
X_test = z_test.drop(columns=columnas_a_eliminar, errors='ignore')
test_ids = X_test['ID']

preds_test_encoded = full_pipeline.predict(X_test)

mapa_inverso_rendimiento = {0: 'bajo', 1: 'medio-bajo', 2: 'medio-alto', 3: 'alto'}
preds_test_texto = [mapa_inverso_rendimiento[p] for p in preds_test_encoded]

submission = pd.DataFrame({'ID': test_ids, 'RENDIMIENTO_GLOBAL': preds_test_texto})

submission.to_csv("my_submission.csv", index=False)

submission.head()

/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(


,ID,RENDIMIENTO_GLOBAL
0,550236,alto
1,98545,medio-alto
2,499179,alto
3,782980,bajo
4,785185,bajo
